- В яких країнах світу населення у 2100 році буде більшим, ніж було у 2000 році, а в яких меншим?

- На скільки відсотків збільшиться або зменшиться населення кожної країни у 2100 році у порівнянні з 2000 роком?

- Яким буде населення країн світу у 2100 році?

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
world_countries = gpd.read_file('world-countries.json')
world_countries.head()

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [3]:
population_df = pd.read_csv('population_prospects.csv')
population_df.head()

,variant,country,country_code,parent_code,year,population
0,Estimates,Burundi,108,910,1950,2308.927
1,Estimates,Comoros,174,910,1950,159.459
2,Estimates,Djibouti,262,910,1950,62.000
3,Estimates,Eritrea,232,910,1950,822.347
4,Estimates,Ethiopia,231,910,1950,18128.030


In [4]:
country_codes = pd.read_csv("country-codes.csv")
country_codes = country_codes.rename(columns={"numeric": "country_code"})
country_codes.head()

,country,alpha2,alpha3,country_code
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20


In [5]:
country_codes.drop(['country', 'alpha2'], axis = 1, inplace = True)
country_codes.head()

,alpha3,country_code
0,AFG,4
1,ALB,8
2,DZA,12
3,ASM,16
4,AND,20


In [6]:
df = population_df.merge(country_codes, how='left', on='country_code')
df.head()

,variant,country,country_code,parent_code,year,population,alpha3
0,Estimates,Burundi,108,910,1950,2308.927,BDI
1,Estimates,Comoros,174,910,1950,159.459,COM
2,Estimates,Djibouti,262,910,1950,62.000,DJI
3,Estimates,Eritrea,232,910,1950,822.347,ERI
4,Estimates,Ethiopia,231,910,1950,18128.030,ETH


In [7]:
df.drop(['parent_code', 'country_code'], axis = 1, inplace = True)
df.columns = ['variant', 'country', 'year', 'population', 'alpha3']
df.head()

,variant,country,year,population,alpha3
0,Estimates,Burundi,1950,2308.927,BDI
1,Estimates,Comoros,1950,159.459,COM
2,Estimates,Djibouti,1950,62.000,DJI
3,Estimates,Eritrea,1950,822.347,ERI
4,Estimates,Ethiopia,1950,18128.030,ETH


In [8]:
estimates_df = df.loc[df['variant'] == 'Estimates']
df_year2000 = estimates_df.loc[df['year'] == 2000].reset_index()

forecasts_df = df.loc[df['variant'] == 'Medium variant']
df_year2100 = forecasts_df.loc[df['year'] == 2100].reset_index()

In [9]:
df_year2000 = df_year2000.rename(columns={"population": "population_2000"})
df_year2000.head()

,index,variant,country,year,population_2000,alpha3
0,11750,Estimates,Burundi,2000,6378.871,BDI
1,11751,Estimates,Comoros,2000,542.358,COM
2,11752,Estimates,Djibouti,2000,717.577,DJI
3,11753,Estimates,Eritrea,2000,2292.413,ERI
4,11754,Estimates,Ethiopia,2000,66224.809,ETH


In [10]:
df_year2100 = df_year2100.rename(columns={"population": "population_2100"})
df_year2100.head()

,index,variant,country,year,population_2100,alpha3
0,35485,Medium variant,Burundi,2100,50904.072,BDI
1,35486,Medium variant,Comoros,2100,2186.509,COM
2,35487,Medium variant,Djibouti,2100,1331.656,DJI
3,35488,Medium variant,Eritrea,2100,9061.777,ERI
4,35489,Medium variant,Ethiopia,2100,294392.903,ETH


In [11]:
df_year2000.drop(["variant", "year", "index"], axis=1, inplace=True)
df_year2100.drop(["variant", "year", "index"], axis=1, inplace=True)

In [12]:
df_merged = df_year2000.merge(df_year2100, on='country')
df_merged.head()

,country,population_2000,alpha3_x,population_2100,alpha3_y
0,Burundi,6378.871,BDI,50904.072,BDI
1,Comoros,542.358,COM,2186.509,COM
2,Djibouti,717.577,DJI,1331.656,DJI
3,Eritrea,2292.413,ERI,9061.777,ERI
4,Ethiopia,66224.809,ETH,294392.903,ETH


In [13]:
df_merged.drop(["alpha3_y"], axis=1, inplace=True)
df_merged = df_merged.rename(columns={"alpha3_x": "country_id"})

In [14]:
df_merged.head()

,country,population_2000,country_id,population_2100
0,Burundi,6378.871,BDI,50904.072
1,Comoros,542.358,COM,2186.509
2,Djibouti,717.577,DJI,1331.656
3,Eritrea,2292.413,ERI,9061.777
4,Ethiopia,66224.809,ETH,294392.903


In [15]:
df_merged['population_diff'] = np.where(df_merged['population_2100'] >= df_merged['population_2000'], 'increase', 'decrease')
df_merged.head()

,country,population_2000,country_id,population_2100,population_diff
0,Burundi,6378.871,BDI,50904.072,increase
1,Comoros,542.358,COM,2186.509,increase
2,Djibouti,717.577,DJI,1331.656,increase
3,Eritrea,2292.413,ERI,9061.777,increase
4,Ethiopia,66224.809,ETH,294392.903,increase


### 1. В яких країнах світу населення у 2100 році буде більшим, ніж було у 2000 році, а в яких меншим?

Для відповіді на це запитання, я вирішила використовувати два кольори у моїй шкалі: зелений та червоний. 
Оскільки нам потрібно показати зміну в кількості населення, то за допомогою зеленого кольору добре зафарбувати країни, в яких населення збільшиться, а за допомогою червоного, в яких зменшиться.

In [23]:
c1 = alt.Chart(world_countries).transform_lookup(
    lookup = 'id',
    from_ = alt.LookupData(data = df_merged, 
                           key = 'country_id',
                           fields=['population_diff', 'country'])
).project(type = 'equalEarth'
).mark_geoshape(stroke = "white").encode(
    color = alt.Color('population_diff:N', 
                      scale = alt.Scale(range = ["green", "red"],
                          domain = ['increase', 'decrease']),
                      title='Population difference'),
    tooltip = [ alt.Tooltip('country:N')]
).properties(title = "In which countries the population in 2100 will be bigger than in 2000?", 
             width = 800, height = 500, background = '#F9F9F9', padding = 25)

In [25]:
c1.show()

Displaying chart at http://localhost:17464/


In [18]:
df_merged['population_diff_val'] =  df_merged["population_2100"] - df_merged["population_2000"]
df_merged['percentage_diff'] =  round(((df_merged["population_diff_val"] /df_merged["population_2000"]) * 100),1)
df_merged.head()

,country,population_2000,country_id,population_2100,population_diff,population_diff_val,percentage_diff
0,Burundi,6378.871,BDI,50904.072,increase,44525.201,698.0
1,Comoros,542.358,COM,2186.509,increase,1644.151,303.1
2,Djibouti,717.577,DJI,1331.656,increase,614.079,85.6
3,Eritrea,2292.413,ERI,9061.777,increase,6769.364,295.3
4,Ethiopia,66224.809,ETH,294392.903,increase,228168.094,344.5


### 2. На скільки відсотків збільшиться або зменшиться населення кожної країни у 2100 році у порівнянні з 2000 роком?

Знову ж таки в цьому запитанні я використовувала шкалу з червоним та зеленим кольорами, проте з більшою кількістю відтінків між крайніми точками.

In [19]:
c2 = alt.Chart(world_countries).transform_lookup(
    lookup = 'id',
    from_ = alt.LookupData(data = df_merged, 
                           key = 'country_id',
                           fields=['percentage_diff', 'country'])
).project(type = 'equalEarth'
).mark_geoshape(stroke = "white").encode(
    color = alt.Color('percentage_diff:Q', 
                      scale = alt.Scale(scheme = 'redyellowgreen'),
                      bin = alt.Bin(step = 200),
                      title='Percentage difference'),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('percentage_diff:N')
    ]
).properties(title="Population percentage change 2001 vs 2100", 
             width = 800, height = 500, background = '#F9F9F9', padding = 25)

In [26]:
c2.show()

Displaying chart at http://localhost:17464/


### 3. Яким буде населення країн світу у 2100 році?

В цьому запитанні я вирішила використовувати максимально контрасну шкалу, щоб краще помітити різницю між різними країнами.

In [21]:
c3 = alt.Chart(world_countries).transform_lookup(
    lookup = 'id',
    from_ = alt.LookupData(data = df_merged, 
                           key = 'country_id',
                           fields=['population_2100', 'country'])
).project(type = 'equalEarth'
).mark_geoshape(stroke = 'white').encode(
    color = alt.Color('population_2100:Q', 
                      scale = alt.Scale(scheme = 'spectral'), 
                      bin = alt.Bin(step = 50000), 
                      title='Population'),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('population_2100:N')
    ]
).properties(title="Population in 2100", 
             width = 800, height = 500, background = '#F9F9F9', padding = 25)

In [27]:
c3.show()

Displaying chart at http://localhost:17464/
